In [2]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import numpy as np
import pandas as pd
import seaborn as sns    
%matplotlib notebook

In [3]:
'''
Loading the dataset into the program
'''

dataset = pd.read_csv("LDOW_2020.csv", encoding='unicode_escape')

In [6]:
dataset.head(2)

,Timestamp,Workshop,Paper title,Link to article,Author-1,Author-1_affiliation,Author-1_country,Author-2,Author-2_affiliation,Author-2_country,...,Author-5_country,Author-6,Author-6_affiliation,Author-6_country,Author-7,Author-7_affiliation,Author-7_country,Paper abstract,Paper Keyword,Paper Categories
0,12/26/2020 20:05:23,2008,Weaving SIOC into the Web of Linked Data,http://events.linkeddata.org/ldow2008/papers/0...,Uldis Bojars,Digital Enterprise Research Institute,Ireland,Alexandre Passant,Electricit de France,France,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Social media sites can act as a rich source of...,NaN,NaN
1,12/26/2020 20:14:32,2008,"SEMANTIC MARC, MARC 21 AND THE SEMANTIC WEB",http://events.linkeddata.org/ldow2008/papers/0...,Rob Styles,Knights Court,United Kingdom,Danny Ayers,Knights Court,United Kingdom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The MARC standard for exchanging bibliographic...,"MARC, MARC21, RDF, Semantic Web, Data Conversi...",NaN


In [12]:
'''
Cleaning the abstract and titles by removing stop words
'''

import nltk
stop_words = nltk.corpus.stopwords.words('english')
wtk = nltk.tokenize.RegexpTokenizer(r'\w+')
wnl = nltk.stem.wordnet.WordNetLemmatizer()
def normalize_corpus(papers):
    norm_papers = []
    for paper in papers:
        paper = paper.lower()
        paper_tokens = [token.strip() for token in wtk.tokenize(paper)]
        paper_tokens = [wnl.lemmatize(token) for token in paper_tokens if not token.isnumeric()]
        paper_tokens = [token for token in paper_tokens if len(token) > 1]
        paper_tokens = [token for token in paper_tokens if token not in stop_words]
        paper_tokens = list(filter(None, paper_tokens))
        if paper_tokens:
            norm_papers.append(paper_tokens)
            
    return norm_papers
    
norm_papers = normalize_corpus(dataset["Paper abstract"])
dataset['Normalised abstract']=normalize_corpus(dataset["Paper abstract"])

def clear_title(title_):
    return title_.replace('\n',' ').lower()
    
dataset['title']=dataset['Paper title'].apply(clear_title)

dataset[['Normalised abstract', 'title']]


,Normalised abstract,title
0,"[social, medium, site, act, rich, source, larg...",weaving sioc into the web of linked data
1,"[marc, standard, exchanging, bibliographic, da...","semantic marc, marc 21 and the semantic web"
2,"[many, institution, grant, access, metadata, r...",the oai2lod server: exposing oai-pmh metadata ...
3,"[availability, large, amount, interlinked, sem...",a case study on linked data generation and con...
4,"[availability, linked, rdf, data, remains, sig...",sparqplug: generating linked data from legacy ...
...,...,...
155,"[paper, present, d2rml, data, rdf, mapping, la...",d2rml: integrating heterogeneous data and web ...
156,"[relational, data, source, still, one, popular...",evaluating approaches for supervised semantic ...
157,"[document, may, include, mention, people, loca...",geek: incremental graph-based entity disambigu...
158,"[internet, user, personal, data, spread, sever...",a knowledge base for personal information mana...


In [8]:
'''
Droping timestamp from the file

'''

dataset.drop(['Timestamp'], 1, inplace=True)

In [10]:
'''
Added the number of authors to the dataset
'''

columns_list=['Author-1', 'Author-2','Author-3','Author-4','Author-5',
             'Author-6','Author-7']

def get_authors(authors):
    c=7
    if(type(authors[0]) == float and np.isnan(authors[0])):
        c=c-1
    if(type(authors[1]) == float and np.isnan(authors[1])):
        c=c-1
    if(type(authors[2]) == float and np.isnan(authors[2])):
        c=c-1
    if(type(authors[3]) == float and np.isnan(authors[3])):
        c=c-1
    if(type(authors[4]) == float and np.isnan(authors[4])):
        c=c-1
    if(type(authors[5]) == float and np.isnan(authors[5])):
        c=c-1
    if(type(authors[6]) == float and np.isnan(authors[6])):
        c=c-1
    return  c

dataset['Number_of_authors']=dataset[columns_list].apply(get_authors, axis=1)

In [13]:
'''
Added the citations to the dataset
'''
citations = pd.read_csv("citations.csv", encoding='latin-1')
citations = citations
citations['title'] = citations['Paper title'].apply(clear_title)
citations = citations[['title','citation']]
final_dataset = dataset.merge(citations, on='title',how='inner')

final_dataset.to_csv('dataset_ldow.csv')

In [15]:
'''
Process to create a RDF dataset for LDOW workshops
'''
import rdflib
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, OWL, RDF, RDFS, SKOS, VOID, XMLNS, XSD

from rdflib import Graph, Literal, RDF, URIRef, BNode

BIBO = rdflib.Namespace("http://purl.org/ontology/bibo/")
SWRC = rdflib.Namespace("http://swrc.ontoware.org/ontology#")
PERSON= rdflib.Namespace("http://data.semanticweb.org/person/")
LDOW = rdflib.Namespace("http://example.org/ldow/")
FOAF = rdflib.Namespace("http://xmlns.com/foaf/0.1/")
DBPEDIA = rdflib.Namespace("http://dbpedia.org/resource/")
VCARD =  rdflib.Namespace("http://www.w3.org/2006/vcard/ns#")
GEONAME =  rdflib.Namespace("http://www.geonames.org/ontology#")


# PREFIX swrc:<http://swrc.ontoware.org/ontology#>
# swj:researcher swrc:aliation http://data.semanticweb.org/organization/uc-santa-barbara
# swc:holdsRole http://data.semanticweb.org/conference/pcmember
# terms:isCreatorOf http://semantic-web-journal.com/sejp/node/204
# bibo:isEditorOf http://semantic-web-journal.com/sejp/node/108
# swj:isReviewerOf http://semantic-web-journal.com/sejp/node/122

event_workshop=  rdflib.URIRef("http://events.linkeddata.org/")
# create a Graph
g = Graph() 
g.bind("bibo", BIBO)
g.bind("dcterms", DCTERMS)
g.bind("swrc", SWRC)
g.bind("ldow", LDOW)
g.bind("person", PERSON)
g.bind("foaf", FOAF)
g.bind("dbpedia",DBPEDIA)
g.bind("vcard",VCARD)
g.bind("geoname",GEONAME)

def add_authors(g, Number_of_authors,index):
    for i in range(1,Number_of_authors+1):
#             print(index,article['countries'][i-1].strip(' ').replace(' ',''))
            person_node=rdflib.term.URIRef(LDOW+'person-'+str(index)+'-authors_'+str(i))
            g.add((authors_node,rdflib.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#'+'_'+str(i)), person_node))
            g.add((person_node,RDF.type, rdflib.URIRef(PERSON)))
            g.add((person_node,FOAF.name, Literal((article['Author-'+str(i)]))))
            g.add((person_node,SWRC.affiliation, Literal((article['Author-'+str(i)+'_affiliation'].strip(' ')))))
            g.add((person_node,GEONAME.name, rdflib.URIRef(DBPEDIA+article['Author-'+str(i)+'_country'].strip(' ').replace(' ',''))))

for index, article in final_dataset.iterrows():
    article_ = rdflib.term.URIRef(LDOW+str(article['Workshop'])+'-'+str(index))
    # Add triples using store's add() method.
#     article_node = LDOW+article['Workshop']+'-'+index

    g.add((article_, RDF.type, rdflib.URIRef(BIBO+'AcademicArticle')))
    workshop = BNode()  # a GUID is generated
    g.add((article_, rdflib.URIRef(SWRC+'atEvent'), workshop)) 
    g.add((workshop, DCTERMS.identifier, event_workshop+'ldow'+str(article['Workshop'])))
    g.add((workshop, RDF.type, rdflib.URIRef(BIBO+'Workshop')))
    g.add((workshop, DCTERMS.title, Literal('LDOW'+str(article['Workshop']))))
    
    g.add((article_, DCTERMS.title,  Literal(article['title'])))
    g.add((article_, rdflib.URIRef(SWRC+'year'), Literal(str(article['Workshop']))))
    authors_node=rdflib.term.URIRef(LDOW+str(article['Workshop'])+'-'+str(index)+'-authorsList')
    g.add((article_, rdflib.URIRef(BIBO+'authorList'),authors_node))
    g.add((article_, rdflib.URIRef(LDOW+'citations'),Literal(article['citation'])))
    g.add((article_, rdflib.URIRef(LDOW+'normalizedAbstract'),Literal(article['Normalised abstract'])))

    if(type(article['Paper Categories'])!=float ):
        keys=article['Paper Categories'].split(';')
        for key in keys:
            g.add((article_, rdflib.URIRef(LDOW+'classification'),Literal(key.strip(' '))))
    
    if(type(article['Paper Keyword'])!=float ):
        keys=article['Paper Keyword'].split(',')
        for key in keys:
            g.add((article_, DCTERMS.subject,Literal(key.strip(' '))))
    g.add((article_, rdflib.URIRef(LDOW+'numberOfAuthors'),Literal(article['Number_of_authors'])))
    add_authors(g,article['Number_of_authors'],index)
    g.add((article_, rdflib.URIRef(SWRC+'pdf'), rdflib.URIRef(article['Link to article'])))

print(g.serialize(format="n3").decode("utf-8"))
g.serialize(destination='LDOWdataset.ttl', format='turtle')



@prefix bibo: <http://purl.org/ontology/bibo/> .
@prefix dbpedia: <http://dbpedia.org/resource/> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix geoname: <http://www.geonames.org/ontology#> .
@prefix ldow: <http://example.org/ldow/> .
@prefix person: <http://data.semanticweb.org/person/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix swrc: <http://swrc.ontoware.org/ontology#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ldow:2008-0 a bibo:AcademicArticle ;
    ldow:citations 43 ;
    ldow:normalizedAbstract "['social', 'medium', 'site', 'act', 'rich', 'source', 'large', 'amount', 'data', 'letting', 'anyone', 'easily', 'create', 'content', 'web', 'sioc', 'ontology', 'tool', 'developed', 'allow', 'express', 'information', 'interlinked', 'rdf', 'data', 'paper', 'describes', 'approach', 'making', 'social', 'web', 'information', 'integral', 'part', 'web', 'linked', 'data']" ;
    ldow:numberOfAuthors 4 ;


In [206]:
q = "select (COUNT(?country) AS ?cnt) where { ?country <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://purl.org/ontology/bibo/AcademicArticle> }"
x = g.query(q)
x1 = g.query(q)
print(list(x1))

[(rdflib.term.Literal('160', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)]


In [16]:
sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    select (COUNT(?country) AS ?cnt) 
    where { ?country rdf:type <http://purl.org/ontology/bibo/AcademicArticle> .}
    """
# q = "select (COUNT(?country) AS ?cnt) where { ?country <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://purl.org/ontology/bibo/AcademicArticle> }"
x = g.query(sparql_query)
print(list(x))

[(rdflib.term.Literal('160', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)]


### Number of papers in a year

In [17]:
sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    prefix swrc: <http://swrc.ontoware.org/ontology#> 
    select ?year (COUNT(?year) AS ?cnt) 
    where { ?article rdf:type <http://purl.org/ontology/bibo/AcademicArticle> .
            ?article swrc:year ?year}
    group by ?year
    order by ?year
    """
x = g.query(sparql_query)
for row in x:
    print(row.year, row.cnt)


2008 23
2009 21
2010 19
2011 12
2012 17
2013 13
2014 13
2015 12
2016 14
2017 8
2018 8


### List of keywords

In [18]:
sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX dcterms: <http://purl.org/dc/terms/> 
    select ?keyword (COUNT(?keyword) AS ?cnt) 
    where { ?article rdf:type <http://purl.org/ontology/bibo/AcademicArticle> .
            ?article dcterms:subject ?keyword}
    group by ?keyword
    order by desc(?cnt) 
    """
x = g.query(sparql_query)
for row in x:
    if(int(row.cnt)>=3):
        print(row.keyword, row.cnt)


Linked Data 58
Semantic Web 23
RDF 18
Linked Open Data 11
linked data 9
SPARQL 6
Provenance 4
Web of Data 4
ontology 4
Linked data 4
URI 4
FOAF 3
Ontology 3
Data Integration 3
Co-reference 3
User Interface 3
Visualization 3


### List of taxonomy

In [19]:
sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX ldow: <http://example.org/ldow/> 
    select ?category (COUNT(?category) AS ?cnt) 
    where { ?article rdf:type <http://purl.org/ontology/bibo/AcademicArticle> .
            ?article ldow:classification ?category}
    group by ?category
    order by desc(?cnt) 
    """
x = g.query(sparql_query)
for row in x:
    if(int(row.cnt)>=1):
        print(row.category, row.cnt)


H.4 [Information Systems Applications]: Miscellaneous 8
H.3.5 [Online Information Services]: Web-based services 6
H.4.m [Information Systems]: Miscellaneous 4
H.3.5 [Information Storage and Retrieval]: Online Information Services 3
H.4 [Information Systems Applications]: General 3
H.3.3 [Information Storage and Retrieval]: Information Search and Retrieval 3
H.3.d [Information Technology and Systems]: Metadata 3
H.5.4 [Hypertext/Hypermedia]: Navigation 3
D.2.12 [Interoperability]: Data mapping 3
H.3.5 [Online Information Services]: Data sharing 3
K.6.5 [Management of Computing and Information Systems]: Security and Protection 2
D.4.3 [Operating Systems]: File Systems Management 2
D.2 [Software]: Software Engineering 2
H.3.1 [Information Storage and Retrieval]: Content Analysis and Indexing 2
H.5.2 [User interfaces]: GUIs, Interaction styles 2
H.3.3 [Information Search and Retrieval]: Information filtering 2
H.4 [Information Systems]: Information Systems Applications 2
I.7.2 [Document Pr

### List of countries

In [20]:
sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bibo: <http://purl.org/ontology/bibo/> 
    PREFIX ldow: <http://example.org/ldow/> 
    PREFIX geoname: <http://www.geonames.org/ontology#>
    select ?country (COUNT(?country) AS ?cnt) (SUM(?numberOfAuthors) AS ?cntAuthor) 
    where { ?article rdf:type <http://purl.org/ontology/bibo/AcademicArticle> .
            ?article bibo:authorList ?authorList.
            ?authorList rdf:_1 ?author.
            ?author geoname:name ?country.
            ?article ldow:numberOfAuthors ?numberOfAuthors.
          }
    group by ?country
    order by desc(?cnt) 
    """
x = g.query(sparql_query)
s=0
for row in x:
    s=s+int(row.cnt)
    if(int(row.cnt)>=1):
        print(row.country, row.cnt, row.cntAuthor)
print(s)

http://dbpedia.org/resource/Germany 30 95
http://dbpedia.org/resource/UnitedKingdom 27 75
http://dbpedia.org/resource/US 17 64
http://dbpedia.org/resource/France 11 34
http://dbpedia.org/resource/Ireland 10 38
http://dbpedia.org/resource/Belgium 10 48
http://dbpedia.org/resource/Austria 10 27
http://dbpedia.org/resource/Italy 6 22
http://dbpedia.org/resource/Canada 6 15
http://dbpedia.org/resource/Spain 5 13
http://dbpedia.org/resource/Switzerland 4 14
http://dbpedia.org/resource/China 4 13
http://dbpedia.org/resource/Australia 3 10
http://dbpedia.org/resource/Czechia 3 9
http://dbpedia.org/resource/India 2 6
http://dbpedia.org/resource/Turkey 2 7
http://dbpedia.org/resource/Greece 2 6
http://dbpedia.org/resource/Brazil 1 2
http://dbpedia.org/resource/Finland 1 3
http://dbpedia.org/resource/Denmark 1 3
http://dbpedia.org/resource/Taiwan 1 5
http://dbpedia.org/resource/Slovenia 1 3
http://dbpedia.org/resource/Japan 1 5
http://dbpedia.org/resource/Chile 1 2
http://dbpedia.org/resource/Cy

### Number of authors per year

In [21]:
sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX swrc: <http://swrc.ontoware.org/ontology#> 
    PREFIX ldow: <http://example.org/ldow/> 
    PREFIX geoname: <http://www.geonames.org/ontology#>
    select ?year (SUM(?numberOfAuthors) AS ?sum_auth) (AVG(?numberOfAuthors) AS ?avg_auth) 
    where { ?article rdf:type <http://purl.org/ontology/bibo/AcademicArticle> .
            ?article swrc:year ?year.
            ?article ldow:numberOfAuthors ?numberOfAuthors.
          }
    group by ?year
    order by asc(?year) 
    """
x = g.query(sparql_query)
for row in x:
    print(row.year, row.sum_auth, round(float(row.avg_auth),2))


2008 65 2.83
2009 54 2.57
2010 62 3.26
2011 33 2.75
2012 59 3.47
2013 45 3.46
2014 53 4.08
2015 46 3.83
2016 54 3.86
2017 24 3.0
2018 26 3.25


### list of affliliations

In [22]:
sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bibo: <http://purl.org/ontology/bibo/> 
    PREFIX swrc: <http://swrc.ontoware.org/ontology#> 
    PREFIX geoname: <http://www.geonames.org/ontology#>
    select ?affiliation (COUNT(?affiliation) AS ?cnt) 
    where { ?article rdf:type <http://purl.org/ontology/bibo/AcademicArticle> .
            ?article bibo:authorList ?authorList.
            ?authorList rdf:_1 ?author.
            ?author swrc:affiliation ?affiliation.
          }
    group by ?affiliation
    order by desc(?cnt) 
    """
x = g.query(sparql_query)
s=0
for row in x:
    s=s+int(row.cnt)
    if(int(row.cnt)>=1):
        print(row.affiliation,';', row.cnt)
print(s)

Ghent University ; 8
University of Southampton ; 7
National University of Ireland ; 5
University of Leipzig ; 4
Digital Enterprise Research Institute ; 3
University of Vienna ; 3
Open University ; 3
University of Edinburgh ; 3
University of Toronto ; 3
Freie Universitt Berlin ; 3
University of Oxford ; 2
Ege University ; 2
IBM ; 2
EPFL ; 2
Renault ; 2
OpenLink Software Inc ; 2
University of Sheffield ; 2
Universitt zu Berlin ; 2
Leipzig University ; 2
University of Fribourg ; 2
Los Alamos National Laboratory ; 2
Universit Paris ; 2
Knights Court ; 2
National Technical University of Athens ; 2
Czech Technical University in Prague ; 2
Freie Universitat Berlin ; 2
University of Freiburg ; 2
University of Georgia ; 1
Institute AIFB ; 1
Indian Institute Of Technology ; 1
ADAPT Centre ; 1
University of Bonn ; 1
Catholic University of Rio de Janeiro ; 1
University of Jyvskyl ; 1
Aalborg University ; 1
Institute of Information Systems & Information Management ; 1
Politecnico di Milano ; 1

### Citation per year

In [23]:
sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bibo: <http://purl.org/ontology/bibo/> 
    PREFIX geoname: <http://www.geonames.org/ontology#>
    PREFIX ldow: <http://example.org/ldow/> 
    PREFIX swrc: <http://swrc.ontoware.org/ontology#> 

    select  ?year (SUM(?citations) as ?citation_total)
    where { ?article rdf:type <http://purl.org/ontology/bibo/AcademicArticle> .
            ?article ldow:citations ?citations.
            ?article swrc:year ?year
          }
    group by ?year
    order by asc(?year) 
    """
x = g.query(sparql_query)
for row in x:
    if(int(row.citation_total)>=1):
        print(row.year, row.citation_total)
# print(list(x))

2008 1400
2009 2057
2010 2433
2011 407
2012 645
2013 433
2014 681
2015 132
2016 149
2017 33
2018 100


### Citation per Author

In [ ]:
sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bibo: <http://purl.org/ontology/bibo/> 
    PREFIX geoname: <http://www.geonames.org/ontology#>
    PREFIX ldow: <http://example.org/ldow/> 
    PREFIX swrc: <http://swrc.ontoware.org/ontology#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/> 

    select  ?name  ?affiliation ?citations ?longevity (ROUND(xsd:float(?citations/?longevity)) as ?NCII) 
    where { ?article rdf:type <http://purl.org/ontology/bibo/AcademicArticle> .
            ?article ldow:citations ?citations.
            ?article bibo:authorList ?authorList.
            ?article swrc:year ?year.
            ?authorList ?p ?author.
            ?author foaf:name ?name.
            ?author swrc:affiliation ?affiliation.

            BIND (((2020 - xsd:integer(?year))) as ?longevity)
          }
    group by ?name
    order by desc(?NCII) 
    """
x = g.query(sparql_query)
for row in x:
    if(int(row.citations)>=1):
        print(row.name,';', row.affiliation,';', row.citations,';', row.longevity,';', row.NCII)
# print(list(x))

### citation per organization

In [216]:
sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bibo: <http://purl.org/ontology/bibo/> 
    PREFIX geoname: <http://www.geonames.org/ontology#>
    PREFIX ldow: <http://example.org/ldow/> 
    PREFIX swrc: <http://swrc.ontoware.org/ontology#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/> 

    select  ?affiliation (SUM(?citations) as ?total_citation) (COUNT(DISTINCT(?article)) AS ?cnt)
    where { ?article rdf:type <http://purl.org/ontology/bibo/AcademicArticle> .
            ?article ldow:citations ?citations.
            ?article bibo:authorList ?authorList.
            ?authorList ?p ?author.
            ?author swrc:affiliation ?affiliation.
          }
    group by ?affiliation
    order by desc(?total_citation) 
    """
x = g.query(sparql_query)
for row in x:
    if(int(row.cnt)>=1):
        print(row.affiliation,';', row.total_citation,';', row.cnt)
# print(list(x))

National University of Ireland ; 3635 ; 11
Multimedia Lab ; 1938 ; 1
University of Southampton ; 1403 ; 9
Freie Universitt Berlin ; 1372 ; 5
Ghent University ; 1360 ; 9
Rensselaer Polytechnic Institute ; 1257 ; 1
Chemnitz University of Technology ; 1014 ; 1
University of Oxford ; 551 ; 4
Freie Universitat Berlin ; 506 ; 2
Talis Ltd. ; 440 ; 1
University of Toronto ; 437 ; 4
Knights Court ; 428 ; 3
Los Alamos National Laboratory ; 389 ; 2
Universit Paris ; 389 ; 3
Digital Enterprise Research Institute ; 355 ; 4
Electricit de France ; 293 ; 2
MIT ; 273 ; 1
University of Leipzig ; 267 ; 4
Universitat zu Berlin ; 256 ; 1
Ege University ; 246 ; 2
IBM ; 222 ; 2
Open University ; 217 ; 3
OpenLink Software Inc ; 209 ; 3
Curtin University of Technology ; 208 ; 2
Old Dominion University ; 207 ; 2
The Stationery Office ; 199 ; 2
EURECOM ; 186 ; 1
University of Vienna ; 183 ; 3
Southeast University ; 172 ; 1
Max Planck Institute for Informatics ; 168 ; 1
SIEMENS ; 167 ; 4
Packard Labs ; 162 ; 1